In [1]:
import pandas as pd
import torch
url ="PS_20174392719_1491204439457_log.csv"
df_actual = pd.read_csv(url, sep=",")
df_actual

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [2]:
df_transactions=df_actual.head(25000)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

print("No of Fraud Transactions:", df_transactions["isFraud"].value_counts()[0])
print("No of Non Fraud Transactions:", df_transactions["isFraud"].value_counts()[1])

print('No Frauds', round(df_transactions['isFraud'].value_counts()[0]/len(df_transactions) * 100,2), '% of the dataset')
print('Frauds', round(df_transactions['isFraud'].value_counts()[1]/len(df_transactions) * 100,2), '% of the dataset')


No of Fraud Transactions: 24917
No of Non Fraud Transactions: 83
No Frauds 99.67 % of the dataset
Frauds 0.33 % of the dataset


In [4]:
df_transactions.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [5]:
from sklearn.preprocessing import LabelEncoder
encoder = {}
for i in df_transactions.select_dtypes('object').columns:
    encoder[i] = LabelEncoder()
    df_transactions[i] = encoder[i].fit_transform(df_transactions[i])

/var/folders/61/3x6_zr2n5nd67km655rj4nd4r4gmsb/T/ipykernel_31500/466741106.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions[i] = encoder[i].fit_transform(df_transactions[i])
/var/folders/61/3x6_zr2n5nd67km655rj4nd4r4gmsb/T/ipykernel_31500/466741106.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transactions[i] = encoder[i].fit_transform(df_transactions[i])
/var/folders/61/3x6_zr2n5nd67km655rj4nd4r4gmsb/T/ipykernel_31500/466741106.py:5: SettingWithCopyWarning: 
A value is trying to

In [6]:
X = df_transactions.drop('isFraud', axis=1)
y = df_transactions['isFraud']

In [7]:
from imblearn.over_sampling import SMOTE

In [8]:
over_sample = SMOTE(random_state=0)
X,y = over_sample.fit_resample(X,y)

In [9]:
y.value_counts()

0    24917
1    24917
Name: isFraud, dtype: int64

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [11]:
X = df_transactions[['step', 'type', 'amount', 'nameOrig','oldbalanceOrg', 'newbalanceOrig','nameDest', 'oldbalanceDest', 'isFlaggedFraud']]
y= df_transactions['isFraud']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
# Turn the values into an array for feeding the classification algorithms.
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [14]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
print(X_test)

[[6.00000000e+00 3.00000000e+00 6.94584000e+03 ... 1.53530000e+04
  0.00000000e+00 0.00000000e+00]
 [8.00000000e+00 1.00000000e+00 2.03546940e+05 ... 7.60000000e+02
  3.78437160e+05 0.00000000e+00]
 [7.00000000e+00 3.00000000e+00 7.28363000e+03 ... 7.87200000e+03
  0.00000000e+00 0.00000000e+00]
 ...
 [8.00000000e+00 1.00000000e+00 5.02507120e+05 ... 2.05400000e+03
  1.53204355e+06 0.00000000e+00]
 [8.00000000e+00 1.00000000e+00 8.13244000e+03 ... 1.38600000e+03
  2.59444167e+06 0.00000000e+00]
 [4.00000000e+00 3.00000000e+00 1.96760000e+02 ... 7.88600000e+03
  0.00000000e+00 0.00000000e+00]]


In [17]:
from typing import Tuple, Union, List
import numpy as np
from sklearn.linear_model import LogisticRegression
import openml

XY = Tuple[np.ndarray, np.ndarray]
Dataset = Tuple[XY, XY]
LogRegParams = Union[XY, Tuple[np.ndarray]]
XYList = List[XY]

def get_model_parameters(model: LogisticRegression) -> LogRegParams:
    """Returns the paramters of a sklearn LogisticRegression model."""
    if model.fit_intercept:
        params = [
            model.coef_,
            model.intercept_,
        ]
    else:
        params = [
            model.coef_,
        ]
    return params


def set_model_params(
    model: LogisticRegression, params: LogRegParams
) -> LogisticRegression:
    """Sets the parameters of a sklean LogisticRegression model."""
    model.coef_ = params[0]
    if model.fit_intercept:
        model.intercept_ = params[1]
    return model

def shuffle(X: np.ndarray, y: np.ndarray) -> XY:
    """Shuffle X and y."""
    rng = np.random.default_rng()
    idx = rng.permutation(len(X))
    return X[idx], y[idx]


def partition(X: np.ndarray, y: np.ndarray, num_partitions: int) -> XYList:
    """Split X and y into a number of partitions."""
    return list(
        zip(np.array_split(X, num_partitions), np.array_split(y, num_partitions))
    )

def set_initial_params(model: LogisticRegression):
    """Sets initial parameters as zeros Required since model params are
    uninitialized until model.fit is called.
    But server asks for initial parameters from clients at launch. Refer
    to sklearn.linear_model.LogisticRegression documentation for more
    information.
    """
    n_classes = 2  
    n_features = 9  # Number of features in dataset
    model.classes_ = np.array([i for i in range(n_classes)])

    model.coef_ = np.zeros((n_classes, n_features))
    if model.fit_intercept:
        model.intercept_ = np.zeros((n_classes,))


In [18]:
import warnings
import flwr as fl
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [19]:
partition_id = np.random.choice(10)
(X_train, y_train) = partition(X_train, y_train, 10)[partition_id]

In [20]:
X_train

array([[7.00000000e+00, 3.00000000e+00, 2.13547000e+04, ...,
        1.12840000e+04, 0.00000000e+00, 0.00000000e+00],
       [8.00000000e+00, 0.00000000e+00, 1.88992440e+05, ...,
        1.32300000e+03, 2.07600000e+04, 0.00000000e+00],
       [7.00000000e+00, 0.00000000e+00, 8.11208000e+04, ...,
        1.75100000e+03, 1.41705270e+05, 0.00000000e+00],
       ...,
       [8.00000000e+00, 3.00000000e+00, 3.21715000e+03, ...,
        4.06700000e+03, 0.00000000e+00, 0.00000000e+00],
       [8.00000000e+00, 4.00000000e+00, 2.22598320e+05, ...,
        4.43000000e+02, 1.44509874e+07, 0.00000000e+00],
       [8.00000000e+00, 3.00000000e+00, 1.35747600e+04, ...,
        3.47400000e+03, 0.00000000e+00, 0.00000000e+00]])

In [21]:
partition_id

3

In [22]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(
        penalty="l2",
        max_iter=1,  # local epoch
        warm_start=True,  # prevent refreshing weights when fitting
    )

model.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1, warm_start=True)

In [23]:
class AML_Detection_Client(fl.client.NumPyClient):
        def get_parameters(self, config):  # type: ignore
            return get_model_parameters(model)

        def fit(self, parameters, config):  # type: ignore
            set_model_params(model, parameters)
            # Ignore convergence failure due to low local epochs
            with warnings.catch_warnings():
                model.fit(X_train, y_train)
                warnings.simplefilter("ignore")
            print(f"Training finished for round {config['server_round']}")
            return get_model_parameters(model), len(X_train), {}

        def evaluate(self, parameters, config):  # type: ignore
            set_model_params(model, parameters)
            loss = log_loss(y_test, model.predict_proba(X_test))
            accuracy = model.score(X_test, y_test)
            print(loss,accuracy)
            return loss, len(X_test), {"accuracy": accuracy}

In [24]:
fl.client.start_numpy_client(server_address="0.0.0.0:8080", client=AML_Detection_Client())

INFO flwr 2023-06-20 11:51:28,912 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-06-20 11:51:28,918 | connection.py:38 | ChannelConnectivity.IDLE
DEBUG flwr 2023-06-20 11:51:28,933 | connection.py:38 | ChannelConnectivity.CONNECTING
DEBUG flwr 2023-06-20 11:51:28,947 | connection.py:38 | ChannelConnectivity.READY
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/line

Training finished for round 1
0.13456297265270425 0.9962666666666666
Training finished for round 2
9.167937977328453 0.7162666666666667
Training finished for round 3
0.3326038065155787 0.9876
Training finished for round 4
1.991991936218432 0.9372
Training finished for round 5
7.897222276432901 0.7714666666666666
